# Chapter 18: Automation and Scripting

Automation transforms Python from a programming language into a powerful tool for eliminating repetitive tasks, orchestrating system operations, and extracting valuable data from the web. Whether you're a system administrator managing hundreds of servers, a data engineer collecting market intelligence, or a developer streamlining your local workflow, Python's rich standard library and ecosystem provide robust tools for reliable automation.

This chapter explores three pillars of Python automation: **system-level scripting** for file and process management, **web scraping** for data extraction, and **task scheduling** for unattended execution. We will emphasize security best practices, error resilience, and modern Python patterns that ensure your automation scripts are maintainable, portable, and production-ready.

## 18.1 System Automation: Interacting with the Operating System

System automation involves programmatically interacting with the file system, executing external commands, and managing environment configurations. Python provides several modules for these tasks, with modern best practices favoring high-level, path-oriented approaches over legacy string-based file manipulation.

### Working with File Paths Using `pathlib`

Introduced in Python 3.4 and now the industry standard, `pathlib` provides an object-oriented interface to the file system that is more intuitive and less error-prone than the older `os.path` module.

```python
from pathlib import Path
from typing import List, Iterator
import shutil

def organize_downloads(download_dir: Path = Path.home() / "Downloads") -> None:
    """
    Organize downloaded files by extension into subdirectories.
    
    Args:
        download_dir: Path to the downloads folder (defaults to ~/Downloads)
    """
    if not download_dir.exists():
        raise FileNotFoundError(f"Directory {download_dir} does not exist")
    
    # Create mapping of extensions to folders
    extension_map: dict[str, Path] = {
        ".pdf": download_dir / "Documents",
        ".jpg": download_dir / "Images",
        ".png": download_dir / "Images",
        ".zip": download_dir / "Archives",
        ".mp4": download_dir / "Videos",
    }
    
    # Ensure destination directories exist
    for dest_dir in set(extension_map.values()):
        dest_dir.mkdir(exist_ok=True)
    
    # Move files
    file_path: Path
    for file_path in download_dir.iterdir():
        if file_path.is_file():
            dest_dir: Path | None = extension_map.get(file_path.suffix.lower())
            if dest_dir:
                try:
                    shutil.move(str(file_path), str(dest_dir / file_path.name))
                    print(f"Moved {file_path.name} to {dest_dir.name}")
                except shutil.Error as e:
                    print(f"Error moving {file_path.name}: {e}")

# Usage
if __name__ == "__main__":
    organize_downloads()
```

**Key `pathlib` Operations:**
*   **Construction**: `Path("/usr/bin")` or `Path.home() / "Documents"`
*   **Inspection**: `.exists()`, `.is_file()`, `.is_dir()`, `.suffix`, `.stem`, `.name`
*   **Navigation**: `.parent`, `.parents[0]`, `.joinpath()`, `/` operator
*   **Operations**: `.mkdir()`, `.touch()`, `.unlink()`, `.rename()`, `.glob("*.py")`

### File I/O Operations

Modern Python emphasizes context managers (`with` statements) for resource management, ensuring files are properly closed even if errors occur.

```python
from pathlib import Path
import json
from typing import Any

def process_log_file(input_path: Path, output_path: Path) -> dict[str, int]:
    """
    Process a log file and generate error statistics.
    
    Uses context managers for safe file handling and JSON for structured output.
    """
    error_counts: dict[str, int] = {}
    
    # Reading with context manager - ensures file closure
    try:
        with open(input_path, 'r', encoding='utf-8') as infile:
            for line_num, line in enumerate(infile, 1):
                line = line.strip()
                if "ERROR" in line:
                    error_type: str = line.split(":")[1].strip() if ":" in line else "Unknown"
                    error_counts[error_type] = error_counts.get(error_type, 0) + 1
                    
    except FileNotFoundError:
        print(f"Error: {input_path} not found")
        raise
    except PermissionError:
        print(f"Error: Permission denied for {input_path}")
        raise
    
    # Writing JSON output with pretty printing
    try:
        with open(output_path, 'w', encoding='utf-8') as outfile:
            json.dump(error_counts, outfile, indent=2, ensure_ascii=False)
    except IOError as e:
        print(f"Failed to write output: {e}")
        raise
    
    return error_counts
```

### Executing System Commands

For executing external programs, modern Python recommends `subprocess` over older modules like `os.system()` due to security and flexibility.

```python
import subprocess
from pathlib import Path
from typing import List, CompletedProcess

def run_system_backup(source_dir: Path, backup_dir: Path) -> bool:
    """
    Create a compressed backup using system tar command.
    
    Demonstrates secure subprocess execution with proper error handling.
    """
    if not source_dir.exists():
        raise ValueError(f"Source directory {source_dir} does not exist")
    
    backup_dir.mkdir(parents=True, exist_ok=True)
    archive_name: Path = backup_dir / f"{source_dir.name}_backup.tar.gz"
    
    # Secure command construction - list prevents shell injection
    cmd: List[str] = [
        "tar", 
        "-czf", 
        str(archive_name), 
        "-C", 
        str(source_dir.parent), 
        str(source_dir.name)
    ]
    
    try:
        # capture_output=True captures stdout/stderr
        # check=True raises CalledProcessError on non-zero exit
        result: CompletedProcess = subprocess.run(
            cmd,
            capture_output=True,
            text=True,
            check=True,
            timeout=300  # 5 minute timeout
        )
        print(f"Backup successful: {archive_name}")
        return True
        
    except subprocess.CalledProcessError as e:
        print(f"Backup failed with exit code {e.returncode}")
        print(f"Error: {e.stderr}")
        return False
        
    except subprocess.TimeoutExpired:
        print("Backup operation timed out")
        return False

# Usage
# run_system_backup(Path("/home/user/documents"), Path("/backups"))
```

**Security Warning:** Never use `shell=True` with user-provided input, as this opens shell injection vulnerabilities. Always pass commands as lists of arguments.

## 18.2 Web Scraping: Extracting Data from the Web

Web scraping programmatically extracts data from websites. While powerful, it requires technical skill, legal awareness, and ethical responsibility. Modern scraping emphasizes respecting robots.txt, rate limiting, and using APIs when available.

### HTTP Requests with `requests`

The `requests` library is the industry standard for HTTP operations, offering a clean API over Python's standard `urllib`.

```python
import requests
from typing import Optional, Dict, Any
import time

class WebScraper:
    """
    Responsible web scraper with rate limiting and error handling.
    """
    
    def __init__(self, delay: float = 1.0, timeout: int = 30) -> None:
        """
        Initialize scraper with politeness settings.
        
        Args:
            delay: Seconds between requests (rate limiting)
            timeout: Request timeout in seconds
        """
        self.delay: float = delay
        self.timeout: int = timeout
        self.session: requests.Session = requests.Session()
        
        # Set headers to identify yourself and accept JSON/HTML
        self.session.headers.update({
            'User-Agent': 'Mozilla/5.0 (compatible; MyBot/1.0; +http://mysite.com/bot)',
            'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
        })
    
    def fetch(self, url: str) -> Optional[str]:
        """
        Fetch HTML content from URL with error handling and rate limiting.
        
        Args:
            url: Target URL
            
        Returns:
            HTML content or None if failed
        """
        try:
            # Respect rate limiting
            time.sleep(self.delay)
            
            response: requests.Response = self.session.get(
                url, 
                timeout=self.timeout
            )
            
            # Check for HTTP errors (4xx, 5xx)
            response.raise_for_status()
            
            return response.text
            
        except requests.exceptions.HTTPError as e:
            print(f"HTTP Error for {url}: {e}")
        except requests.exceptions.ConnectionError:
            print(f"Connection Error for {url}")
        except requests.exceptions.Timeout:
            print(f"Timeout Error for {url}")
        except requests.exceptions.RequestException as e:
            print(f"Request failed for {url}: {e}")
        
        return None
    
    def fetch_json(self, url: str) -> Optional[Dict[str, Any]]:
        """Fetch and parse JSON API response."""
        html: Optional[str] = self.fetch(url)
        if html:
            try:
                return self.session.get(url).json()
            except ValueError:
                print(f"Invalid JSON from {url}")
        return None
    
    def close(self) -> None:
        """Clean up session resources."""
        self.session.close()

# Usage example
if __name__ == "__main__":
    scraper: WebScraper = WebScraper(delay=2.0)  # Be polite, wait 2 seconds
    
    # Example: Fetch a page (replace with actual URL)
    # content: Optional[str] = scraper.fetch("https://example.com")
    
    scraper.close()
```

**Ethical Scraping Guidelines:**
1.  **Check `robots.txt`**: Always verify `/robots.txt` to see which paths are allowed.
2.  **Rate Limiting**: Never hammer servers. Use `time.sleep()` between requests.
3.  **User-Agent**: Identify yourself so webmasters can contact you if issues arise.
4.  **Terms of Service**: Review website ToS; some prohibit scraping entirely.
5.  **APIs First**: Always prefer official APIs over scraping HTML.

### HTML Parsing with BeautifulSoup

Once you fetch HTML, you need to extract specific data. `BeautifulSoup` (from the `bs4` package) is the standard library for parsing HTML and XML documents.

```python
from bs4 import BeautifulSoup, Tag
from typing import List, Optional, Dict
import requests

class ProductExtractor:
    """
    Extract product information from e-commerce HTML.
    Demonstrates BeautifulSoup parsing techniques.
    """
    
    def __init__(self, html_content: str) -> None:
        """
        Initialize parser with HTML content.
        
        Args:
            html_content: Raw HTML string to parse
        """
        # Use 'lxml' parser for speed (requires: pip install lxml)
        # Fallback to 'html.parser' (built-in) if lxml not available
        self.soup: BeautifulSoup = BeautifulSoup(html_content, 'lxml')
    
    def extract_title(self) -> Optional[str]:
        """
        Extract page title.
        
        Returns:
            Title text or None if not found
        """
        # Find returns the first match or None
        title_tag: Optional[Tag] = self.soup.find('title')
        return title_tag.get_text(strip=True) if title_tag else None
    
    def extract_products(self) -> List[Dict[str, str]]:
        """
        Extract all products from a listing page.
        Assumes HTML structure with class names.
        
        Returns:
            List of product dictionaries
        """
        products: List[Dict[str, str]] = []
        
        # find_all returns a list of all matching tags
        # Using CSS selectors via select() is often more powerful
        product_cards: List[Tag] = self.soup.find_all('div', class_='product-card')
        
        for card in product_cards:
            try:
                # Navigate the DOM tree
                name_tag: Optional[Tag] = card.find('h2', class_='product-name')
                price_tag: Optional[Tag] = card.find('span', class_='price')
                link_tag: Optional[Tag] = card.find('a')
                
                if name_tag and price_tag:
                    product: Dict[str, str] = {
                        'name': name_tag.get_text(strip=True),
                        'price': price_tag.get_text(strip=True),
                        'url': link_tag['href'] if link_tag and link_tag.has_attr('href') else ''
                    }
                    products.append(product)
                    
            except Exception as e:
                # Log error but continue processing other products
                print(f"Error parsing product: {e}")
                continue
        
        return products
    
    def extract_tables(self) -> List[List[List[str]]]:
        """
        Extract all HTML tables as structured data.
        
        Returns:
            List of tables, each being a list of rows (lists of cell texts)
        """
        tables: List[List[List[str]]] = []
        
        for table in self.soup.find_all('table'):
            table_data: List[List[str]] = []
            
            # Handle both <thead> and <tbody> rows
            rows: List[Tag] = table.find_all('tr')
            
            for row in rows:
                # Extract all cell types (th and td)
                cells: List[Tag] = row.find_all(['td', 'th'])
                row_data: List[str] = [cell.get_text(strip=True) for cell in cells]
                
                if row_data:  # Skip empty rows
                    table_data.append(row_data)
            
            if table_data:
                tables.append(table_data)
        
        return tables

# Practical Usage Example
if __name__ == "__main__":
    # Example HTML (in real use, this comes from requests.get().text)
    sample_html: str = """
    <html>
        <head><title>Tech Store</title></head>
        <body>
            <div class="product-card">
                <h2 class="product-name">Laptop Pro</h2>
                <span class="price">$999</span>
                <a href="/products/laptop-pro">View</a>
            </div>
            <div class="product-card">
                <h2 class="product-name">Wireless Mouse</h2>
                <span class="price">$29</span>
                <a href="/products/mouse">View</a>
            </div>
        </body>
    </html>
    """
    
    extractor: ProductExtractor = ProductExtractor(sample_html)
    
    # Extract title
    title: Optional[str] = extractor.extract_title()
    print(f"Page Title: {title}")
    
    # Extract products
    products: List[Dict[str, str]] = extractor.extract_products()
    for product in products:
        print(f"Product: {product['name']} - {product['price']}")
```

**BeautifulSoup Best Practices:**
1.  **Always specify a parser**: `'lxml'` is fastest, `'html.parser'` is built-in, `'html5lib'` is most lenient with broken HTML.
2.  **Use `get_text(strip=True)`**: Removes leading/trailing whitespace and normalizes text extraction.
3.  **Check for None**: `find()` returns `None` if no match; always check before accessing attributes.
4.  **CSS Selectors**: Use `.select()` for complex queries (e.g., `soup.select("div.content > p:first-child")`).

## 18.3 Scheduling: Automating Task Execution

Automation requires not just performing tasks, but performing them at specific times or intervals. Python offers several approaches from simple in-process schedulers to enterprise-grade job queues.

### Simple Scheduling with `schedule`

The `schedule` library (third-party, `pip install schedule`) provides a readable, human-friendly API for lightweight automation scripts that run continuously.

```python
import schedule
import time
import logging
from datetime import datetime
from pathlib import Path
import shutil
from typing import Callable

# Configure logging for production visibility
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler('automation.log'),
        logging.StreamHandler()
    ]
)
logger: logging.Logger = logging.getLogger(__name__)

class DataPipeline:
    """
    Automated data processing pipeline with scheduling capabilities.
    """
    
    def __init__(self, source_dir: Path, backup_dir: Path) -> None:
        self.source_dir: Path = source_dir
        self.backup_dir: Path = backup_dir
        self.processed_count: int = 0
    
    def backup_data(self) -> None:
        """Task 1: Backup critical data."""
        try:
            timestamp: str = datetime.now().strftime("%Y%m%d_%H%M%S")
            backup_path: Path = self.backup_dir / f"backup_{timestamp}"
            
            if self.source_dir.exists():
                shutil.copytree(self.source_dir, backup_path)
                logger.info(f"Backup completed: {backup_path}")
            else:
                logger.warning(f"Source directory {self.source_dir} does not exist")
                
        except Exception as e:
            logger.error(f"Backup failed: {e}", exc_info=True)
    
    def generate_report(self) -> None:
        """Task 2: Generate daily analytics report."""
        try:
            logger.info("Generating daily report...")
            # Simulate report generation
            report_path: Path = Path("daily_report.txt")
            report_path.write_text(f"Report generated at {datetime.now()}")
            self.processed_count += 1
            logger.info(f"Report saved to {report_path}")
        except Exception as e:
            logger.error(f"Report generation failed: {e}")

def setup_scheduler(pipeline: DataPipeline) -> None:
    """
    Configure the job scheduler with various timing patterns.
    
    Args:
        pipeline: The DataPipeline instance to schedule tasks for
    """
    # Daily backup at 2:00 AM
    schedule.every().day.at("02:00").do(pipeline.backup_data)
    
    # Report generation every 30 minutes during business hours
    schedule.every(30).minutes.do(pipeline.generate_report)
    
    # Weekly full cleanup on Sundays at 3:00 AM
    schedule.every().sunday.at("03:00").do(lambda: logger.info("Weekly maintenance"))
    
    # Specific time with tags for identification
    schedule.every().hour.do(
        lambda: logger.info("Hourly health check")
    ).tag("monitoring", "health")

def run_scheduler() -> None:
    """Main loop to keep the scheduler running."""
    logger.info("Starting automation scheduler...")
    
    while True:
        try:
            # Run pending jobs
            schedule.run_pending()
            
            # Sleep to prevent CPU spinning (minimum 1 second)
            time.sleep(1)
            
        except KeyboardInterrupt:
            logger.info("Scheduler stopped by user")
            break
        except Exception as e:
            logger.error(f"Scheduler error: {e}")
            # Continue running despite errors
            time.sleep(5)

# Production setup
if __name__ == "__main__":
    pipeline: DataPipeline = DataPipeline(
        source_dir=Path("/data/source"),
        backup_dir=Path("/data/backups")
    )
    
    setup_scheduler(pipeline)
    run_scheduler()
```

**Key Scheduling Patterns:**
*   **Intervals**: `every(10).minutes`, `every(2).hours`, `every().monday`
*   **Specific times**: `every().day.at("10:30")` (24h format)
*   **Tagging**: `.tag("maintenance")` allows canceling specific job types: `schedule.clear('maintenance')`

**Production Considerations:**
The `schedule` library is suitable for long-running single-process scripts but lacks persistence (jobs are lost if the script crashes). For mission-critical automation, use **APScheduler** (Advanced Python Scheduler) or **Celery** with a message broker.

### Enterprise Scheduling with APScheduler

For production environments requiring persistence, multiple job stores, and distributed execution:

```python
from apscheduler.schedulers.background import BackgroundScheduler
from apscheduler.triggers.cron import CronTrigger
from apscheduler.triggers.interval import IntervalTrigger
from apscheduler.events import EVENT_JOB_ERROR, EVENT_JOB_EXECUTED
from datetime import datetime
import logging

logging.basicConfig()
logging.getLogger('apscheduler').setLevel(logging.DEBUG)

def job_listener(event):
    """Callback for job events."""
    if event.exception:
        print(f"Job crashed: {event.job_id}")
    else:
        print(f"Job executed successfully: {event.job_id}")

# Background scheduler (doesn't block main thread)
scheduler: BackgroundScheduler = BackgroundScheduler()

# Add listener
scheduler.add_listener(job_listener, EVENT_JOB_ERROR | EVENT_JOB_EXECUTED)

# Cron trigger (Unix cron-like scheduling)
# Run at 9:30 AM on Mondays and Wednesdays
scheduler.add_job(
    lambda: print("Weekly report triggered"),
    trigger=CronTrigger(day_of_week='mon,wed', hour=9, minute=30),
    id='weekly_report',
    replace_existing=True
)

# Interval trigger with jitter to prevent thundering herd
scheduler.add_job(
    lambda: print("Health check"),
    trigger=IntervalTrigger(minutes=5, jitter=60),  # ±60 seconds randomization
    id='health_check'
)

# Start scheduler
scheduler.start()

# Keep main thread alive
try:
    while True:
        time.sleep(1)
except KeyboardInterrupt:
    scheduler.shutdown()
```

**APScheduler Features:**
*   **Persistence**: Store jobs in SQLAlchemy databases, Redis, or MongoDB
*   **Distributed**: Multiple workers can pull from a shared job store
*   **Triggers**: Cron-like, interval-based, or date-based execution
*   **Job Management**: Add, remove, pause, and reschedule jobs dynamically

## 18.3 Advanced Automation Patterns

### Error Resilience and Idempotency

Production automation must handle failures gracefully and be **idempotent** (running the script multiple times produces the same result as running it once).

```python
from pathlib import Path
import hashlib
import pickle
from typing import Set
from functools import wraps

class IdempotentProcessor:
    """
    Ensures files are processed only once using state tracking.
    """
    
    def __init__(self, state_file: Path = Path(".processor_state")) -> None:
        self.state_file: Path = state_file
        self.processed_hashes: Set[str] = self._load_state()
    
    def _load_state(self) -> Set[str]:
        """Load set of processed file hashes."""
        if self.state_file.exists():
            with open(self.state_file, 'rb') as f:
                return pickle.load(f)
        return set()
    
    def _save_state(self) -> None:
        """Persist state to disk."""
        with open(self.state_file, 'wb') as f:
            pickle.dump(self.processed_hashes, f)
    
    def _get_file_hash(self, filepath: Path) -> str:
        """Generate MD5 hash of file content."""
        hasher = hashlib.md5()
        with open(filepath, 'rb') as f:
            for chunk in iter(lambda: f.read(4096), b""):
                hasher.update(chunk)
        return hasher.hexdigest()
    
    def process_file(self, filepath: Path) -> bool:
        """
        Process file only if not previously processed.
        
        Returns:
            True if processed, False if skipped (already processed)
        """
        file_hash: str = self._get_file_hash(filepath)
        
        if file_hash in self.processed_hashes:
            print(f"Skipping {filepath} (already processed)")
            return False
        
        # Process the file here
        print(f"Processing {filepath}...")
        
        # Mark as processed and save state
        self.processed_hashes.add(file_hash)
        self._save_state()
        return True

# Usage
processor: IdempotentProcessor = IdempotentProcessor()
for file in Path("/data/incoming").glob("*.csv"):
    processor.process_file(file)
```

### Context Managers for Resource Safety

Automation scripts often acquire resources (files, network connections, locks) that must be released. Context managers ensure cleanup happens even if errors occur.

```python
from contextlib import contextmanager
from typing import Generator, Optional
import tempfile
import shutil

@contextmanager
def managed_temp_directory() -> Generator[Path, None, None]:
    """
    Context manager that creates a temp directory and cleans it up.
    Yields the path, then deletes it regardless of exceptions.
    """
    temp_dir: Path = Path(tempfile.mkdtemp())
    print(f"Created temp directory: {temp_dir}")
    
    try:
        yield temp_dir
    finally:
        # Cleanup happens even if exception occurred
        shutil.rmtree(temp_dir)
        print(f"Cleaned up temp directory: {temp_dir}")

# Usage
with managed_temp_directory() as tmp:
    # Work with temporary files
    data_file: Path = tmp / "data.txt"
    data_file.write_text("Sensitive temporary data")
    # Process data...
    print(f"File exists: {data_file.exists()}")
# After exiting 'with' block, directory is deleted
```

## Summary

This chapter equipped you with the tools to automate the world around you using Python. You learned to navigate the file system safely using **`pathlib`**, execute system commands securely with **`subprocess`**, and build resilient automation loops with **`schedule`** and **APScheduler**.

In web scraping, you mastered the **`requests`** library for HTTP communication and **BeautifulSoup** for HTML parsing, while understanding the critical importance of rate limiting, user-agent identification, and legal compliance. You explored advanced patterns including **idempotent processing** to prevent duplicate work and **context managers** for resource-safe automation.

As you move from writing scripts to building distributable applications, managing dependencies becomes crucial. The next chapter explores modern Python dependency management, virtual environments, and packaging standards that ensure your automation scripts and applications run consistently across development, testing, and production environments.

**Next Chapter**: Chapter 19: Dependency Management and Virtual Environments.